# Relatório Parcial - Maratona de Filmes

### Autor: José Rafael Fernandes
---

### Objetivo

O projeto Maratona de Filmes consiste na criação de um algoritmo para maximizar o número de filmes assistidos em um único dia. Vale ressaltar que o projeto conta com duas implementações diferentes para o algoritmo, utilizando diferentes heurísticas. A primeira implementação utiliza o algoritmo guloso, enquanto a segunda utiliza o algoritmo guloso aleatorizado. O objetivo do relatório é comparar os resultados obtidos por cada algoritmo, bem como analisar a influência de cada heurística utilizada.

### Descrição do Problema

Como citado anteriormente, o algoritmo tem como objetivo maximizar o número de filmes assistidos em um único dia. Entretanto, existem algumas restrições que devem ser levadas em consideração. A primeira delas é que cada filme pertence a uma categoria e cada categoria tem um máximo de filmes que podem ser assistidos. Além disso, cada filme possui um tempo de duração, e não se pode começar a assistir outro filme antes de terminar o anterior. Por fim, o algoritmo deve ser capaz de receber um arquivo .txt de entrada, contendo as informações dos filmes, e retornar um arquivo .txt de saída, contendo o número de filmes assistidos e a lista de filmes assistidos. Abaixo, é possível ver um exemplo de arquivo de entrada e de saída.

##### Arquivo de Entrada

``` txt
10 4
1 3 1 2 
11 13 3
14 15 3
10 16 2
10 14 1
11 17 2
11 14 3
13 15 3
14 15 1
12 16 4
12 13 4
```

##### Arquivo de Saída

``` txt
2
11 13 3
14 15 1
```


## Implementação

A ideia da implementação pode ser resumida nos seguintes passos:
- Ler o arquivo de entrada e armazenar as informações em estruturas de dados;
- Ordenar os filmes por horario de término;
- Excluir os filmes inválidos (terminam em outro dia ou terminam e começam no mesmo horário);
- Escolher os filmes que serão assistidos;
    - Para a implementação da heurística gulosa, escolhemos sempre o filme que termina mais cedo, desde que não ultrapasse o limite de filmes por categoria e seu período de exibição esteja disponível;
    - Para a implementação da heurística gulosa aleatorizada, existe 25% de chance de escolhermos um filme aleatório da lista, obedencendo as restrições, e 75% de chance de utilizarmos a estratégia gulosa;
- Escrever o arquivo de saída com a quantidade de filmes assistidos e a lista de filmes assistidos.

Seguindo essa ideia, a implementação dos algoritmos foi feita em C++ e dividida em uma série de funções que realizam tarefas específicas. A seguir, todas as funções são descritas e comentadas:

### Algortimo Guloso

Antes da implementação das funções do algoritmo, é necessário definir uma estrutura de dados que será essencial para o algoritmo. A estrutura de dados escolhida foi a seguinte:

``` cpp
struct movie {
    int startTime;
    int endTime;
    int category;
};
```

A estrutura de dados acima representa um filme, contendo o horário de início, o horário de término e a categoria do filme. A seguir, foi necessária a criação de uma função para conferir se os argumentos passados para o programa são válidos. A função utilizada para conferir os argumentos foi a seguinte:

``` cpp
bool validateArgs(int argc, char *argv[]) {
    if (argc < 2) {
        cerr << "Usage: " << argv[0] << " <filename>" << endl;
        return false;
    }
    return true;
}
```

Depois disso, uma função para abrir o arquivo de entrada e verificar possíveis erros nesse processo foi criada.

``` cpp
bool openFile(const char* filename, ifstream& infile) {
    // Open the file for reading
    infile.open(filename);

    // Check if the file was opened successfully
    if (!infile.is_open()) {
        cerr << "Error: could not open file " << filename << endl;
        return false;
    }

    return true;
}
```

A função acima recebe como argumento o nome do arquivo de entrada e um objeto do tipo ifstream. A função retorna true caso o arquivo seja aberto com sucesso e false caso contrário. A seguir, uma função para ordenar os filmes de acordo com o horário de término foi criada.

``` cpp
vector<movie> sortMoviesByEndTime(vector<movie>& movies) {
    sort(movies.begin(), movies.end(), [](const movie& a, const movie& b) {
        return a.endTime < b.endTime;
    });
    return movies;
}
``` 

A função acima recebe como argumento um vetor de filmes e retorna um vetor de filmes ordenado de acordo com o horário de término. Após isso, uma função para remover filmes invalidos foi criada.

``` cpp
vector<movie> filterMovies(vector<movie>& movies) {
    for (int i = 0; i < movies.size(); i++) {
        if (movies[i].endTime < movies[i].startTime || movies[i].startTime == movies[i].endTime) {
            movies.erase(movies.begin() + i);
            i--;
        }
    }
    return movies;
}
```

A função acima recebe como argumento um vetor de filmes e retorna um vetor de filmes sem filmes inválidos. Um filme é considerado inválido quando o horário de término é menor que o horário de início ou quando o horário de início e o horário de término são iguais. Depois, uma função para escolher os filmes a serem assistidos foi criada.

``` cpp
vector<movie> chooseMovies(vector<movie>& movies, vector<int>& categoriesMax){
    bitset<24> movieTimesOccupied;
    vector<movie> chosenMovies;

    for (int i = 0; i < movies.size(); i++) {
        if (checkIfMovieFits(movieTimesOccupied, movies[i]) && categoriesMax[movies[i].category-1] > 0) {
            chosenMovies.push_back(movies[i]);
            movieTimesOccupied = occupiedTimes(movieTimesOccupied, movies[i]);
            categoriesMax[movies[i].category-1]--;
        }
        if (categoriesMax == vector<int>(categoriesMax.size(), 0)) {
            break;
        }
    }

    return chosenMovies;
} 
```

A função acima recebe como argumento um vetor de filmes e um vetor de inteiros. O vetor de filmes representa os filmes que podem ser assistidos e o vetor de inteiros representa o número máximo de filmes que podem ser assistidos de cada categoria. A função retorna um vetor de filmes contendo os filmes que foram escolhidos para serem assistidos. A função acima é a função principal do algoritmo guloso. Vale ressaltar que a função acima utiliza outras funções auxiliares, que serão descritas a seguir.

``` cpp
bitset<24> occupiedTimes(bitset<24>& times, movie movie){
    for (int i = movie.startTime; i < movie.endTime; i++) {
        times[i] = 1;
    }
    return times;
}

bool checkIfMovieFits(bitset<24>& times, movie movie){
    for (int i = movie.startTime; i < movie.endTime; i++) {
        if (times[i] == 1) {
            return false;
        }
    }
    return true;
}
```

A função occupiedTimes recebe como argumento um bitset de 24 posições (representando as horas do dia) e um filme. A função retorna um bitset de 24 posições com os horários ocupados pelo filme passado como argumento. A função checkIfMovieFits recebe como argumento um bitset de 24 posições e um filme. A função retorna true caso o filme passado como argumento caiba no horário disponível e false caso contrário.

Por fim, todas as funções citadas acimas foram utilizadas na função main para a implementação do algoritmo guloso.

``` cpp
int main(int argc, char *argv[]) {
    // Validate the command line arguments
    if (!validateArgs(argc, argv)) {
        return 1;
    }

    ifstream infile;
    // Check if the file was opened successfully
    if (!openFile(argv[1], infile)) {
        return 1;
    }

    // Create the variables to store the data
    int N;
    int M;
    vector<movie> movies;
    vector<int> categoriesMax;

    // Read the first line of the file and store the first int in N and the second in M
    infile >> N >> M;

    // Read the second line of the file and store the ints in categoriesMax
    for (int i = 0; i < M; i++) {
        int categoryMax;
        infile >> categoryMax;
        categoriesMax.push_back(categoryMax);
    }

    // Read the rest of the file and store the ints of each line in movies
    for (int i = 0; i < N; i++) {
        movie movie;
        infile >> movie.startTime >> movie.endTime >> movie.category;
        movies.push_back(movie);
    }

    // Close the file
    infile.close();


    sortMoviesByEndTime(movies);

    filterMovies(movies);
    
    vector<movie> chosenMovies = chooseMovies(movies, categoriesMax);
    
    cout << chosenMovies.size() << endl;
    for (int i = 0; i < chosenMovies.size(); i++) {
        cout << chosenMovies[i].startTime << " " << chosenMovies[i].endTime << " " << chosenMovies[i].category << endl;
    }
    
    return 0;
}
```

### Algoritmo Guloso Aleatorizado

O algoritmo guloso aleatorizado é uma variação do algoritmo guloso. A diferença entre os dois algoritmos é que o algoritmo guloso aleatorizado possui 25% de chance de escolher aleatoriamente entre os filmes que passam no teste de tempo. Para a implementação dessa alteração, a função chooseMovies foi alterada.

``` cpp
vector<movie> chooseMovies(vector<movie>& movies, vector<int>& categoriesMax){
    bitset<24> movieTimesOccupied;
    vector<movie> chosenMovies;

    int seed = 10;
    char *SEED_VAR = getenv("SEED");
    if (SEED_VAR != NULL)
    {
        seed = atoi(SEED_VAR);
    }
    default_random_engine generator(seed);
    uniform_real_distribution<double> distribution(0.0, 1.0);


    for (int i = 0; i < movies.size(); i++) {
        double rnd_num = distribution(generator); 
        if (rnd_num <= 0.25) {
            int randomMovieIndex = rand() % movies.size();
            if (checkIfMovieFits(movieTimesOccupied, movies[randomMovieIndex]) && categoriesMax[movies[randomMovieIndex].category-1] > 0) {
                chosenMovies.push_back(movies[randomMovieIndex]);
                movieTimesOccupied = occupiedTimes(movieTimesOccupied, movies[randomMovieIndex]);
                categoriesMax[movies[randomMovieIndex].category-1]--;
            }
        }
        else {
            if (checkIfMovieFits(movieTimesOccupied, movies[i]) && categoriesMax[movies[i].category-1] > 0) {
                chosenMovies.push_back(movies[i]);
                movieTimesOccupied = occupiedTimes(movieTimesOccupied, movies[i]);
                categoriesMax[movies[i].category-1]--;

            }
        }
        if (categoriesMax == vector<int>(categoriesMax.size(), 0)) {
            break;
        }
    }

    return chosenMovies;
} 
``` 

## Profiling

Depois de implementar os algoritmos, foi necessário realizar o profiling dos mesmos. Assim, a ferramenta *valgrind* foi utilizada, com o objetivo de entender o comportamento de memória dos algoritmos e possíveis áreas de otimização. O comando utilizado para o profiling foi o seguinte:

``` bash
valgrind --tool=callgrind ./{programa utilizado} inputs/in-10000-6.txt
callgrind_annotate callgrind.out.{pid do valgrind} {codigo fonte do programa}
```

Abaixo, estão os resultados do profiling dos algoritmos.

### Algoritmo Guloso

``` bash

```

### Algoritmo Guloso Aleatorizado

``` bash

```

### Análise dos Resultados de Profiling

## Comparação dos Algoritmos

Para a comparação dos algoritmos vamos utilizar duas perspectivas diferentes. A primeira delas é em relação ao tempo de execução em função do aumento de filmes e categorias. A segunda é em relação ao número de filmes escolhidos em função do aumento de filmes e categorias.

Para isso, vamos utilizar os inputs gerados pelo script fornecido *geradorInputs.cpp* e executar os algoritmos com cada um dos inputs gerados. Os resultados serão armazenados em arquivos de texto, que serão utilizados para gerar os gráficos. Para a geração dos arquivos de entrada, o seguinte script em python foi utilizado:

``` python
varia_filmes = [1000, 10000, 100000, 1000000, 10000000]
varia_categorias = [1, 2, 3, 4, 5, 6]

for i in varia_filmes:
    for j in varia_categorias:
        !./geradorInputs {i} {j}
        !mv input.txt in-{i}-{j}.txt
```

Que resultou em 30 arquivos de entrada, cada um com um número de filmes e categorias diferente.

### Tempo de Execução